***GENERATED CODE FOR customeracquisitions PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '3275', 'mean': '', 'stddev': '', 'min': 'ARVADA', 'max': 'WHEAT RIDGE', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product_Category', 'transformation_label': 'String Indexer'}], 'feature': 'Product_Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3275', 'mean': '', 'stddev': '', 'min': 'Furniture', 'max': 'Technology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product_Category'}, {'feature_label': 'Product_Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product_Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product_Sub-Category', 'transformation_label': 'String Indexer'}], 'feature': 'Product_Sub-Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3275', 'mean': '', 'stddev': '', 'min': 'Appliances', 'max': 'Telephones and Communication', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product_Sub-Category'}, {'feature_label': 'Product_Sub-Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product_Sub-Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer Segment', 'transformation_label': 'String Indexer'}], 'feature': 'Customer Segment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3275', 'mean': '', 'stddev': '', 'min': 'Consumer', 'max': 'Small Business', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer Segment'}, {'feature_label': 'Customer Segment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer Segment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'First Name', 'transformation_label': 'String Indexer'}], 'feature': 'First Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3275', 'mean': '', 'stddev': '', 'min': 'AAXXX', 'max': 'ZEXXX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'First Name'}, {'feature_label': 'First Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('First Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Last Name', 'transformation_label': 'String Indexer'}], 'feature': 'Last Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3275', 'mean': '', 'stddev': '', 'min': 'ABXXXX', 'max': 'ZLXX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Last Name'}, {'feature_label': 'Last Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Last Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Address', 'transformation_label': 'String Indexer'}], 'feature': 'Address', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3275', 'mean': '', 'stddev': '', 'min': '10 MEADOW ROSE LN', 'max': '9956 CLARK DR', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Address'}, {'feature_label': 'Address', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Address')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3275', 'mean': '', 'stddev': '', 'min': 'CO', 'max': 'CO', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MOSAIC HOUSEHOLD', 'transformation_label': 'String Indexer'}], 'feature': 'MOSAIC HOUSEHOLD', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3275', 'mean': '', 'stddev': '', 'min': 'A01', 'max': 'U00', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MOSAIC HOUSEHOLD'}, {'feature_label': 'MOSAIC HOUSEHOLD', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MOSAIC HOUSEHOLD')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MOSAIC DESCRIPTION', 'transformation_label': 'String Indexer'}], 'feature': 'MOSAIC DESCRIPTION', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3275', 'mean': '', 'stddev': '', 'min': 'Aspirational Fusion', 'max': 'Young City Solos', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MOSAIC DESCRIPTION'}, {'feature_label': 'MOSAIC DESCRIPTION', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MOSAIC DESCRIPTION')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Channel', 'transformation_label': 'String Indexer'}], 'feature': 'Channel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3275', 'mean': '', 'stddev': '', 'min': 'Catalog', 'max': 'eCommerce', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Channel'}, {'feature_label': 'Channel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Channel')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run customeracquisitionsHooks.ipynb
try:
	#sourcePreExecutionHook()

	customeracquisitiontrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/demo/Customer_Acquisition/CustomerAcquisitionTrain.csv', 'filename': 'CustomerAcquisitionTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(customeracquisitiontrain)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run customeracquisitionsHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(customeracquisitiontrain,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3275", "mean": "", "stddev": "", "min": "ARVADA", "max": "WHEAT RIDGE", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "Product_Category", "transformation_label": "String Indexer"}], "feature": "Product_Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3275", "mean": "", "stddev": "", "min": "Furniture", "max": "Technology", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product_Category"}, {"transformationsData": [{"feature_label": "Product_Sub-Category", "transformation_label": "String Indexer"}], "feature": "Product_Sub-Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3275", "mean": "", "stddev": "", "min": "Appliances", "max": "Telephones and Communication", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product_Sub-Category"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3275", "mean": "1.08", "stddev": "0.3", "min": "1", "max": "4", "missing": "0"}, "updatedLabel": "Count"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3275", "mean": "1750.41", "stddev": "949.64", "min": "2", "max": "3402", "missing": "0"}, "updatedLabel": "Customer ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store Number", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3275", "mean": "102.35", "stddev": "1.81", "min": "100", "max": "105", "missing": "0"}, "updatedLabel": "Store Number"}, {"transformationsData": [{"feature_label": "Customer Segment", "transformation_label": "String Indexer"}], "feature": "Customer Segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3275", "mean": "", "stddev": "", "min": "Consumer", "max": "Small Business", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer Segment"}, {"transformationsData": [{"feature_label": "First Name", "transformation_label": "String Indexer"}], "feature": "First Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3275", "mean": "", "stddev": "", "min": "AAXXX", "max": "ZEXXX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "First Name"}, {"transformationsData": [{"feature_label": "Last Name", "transformation_label": "String Indexer"}], "feature": "Last Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3275", "mean": "", "stddev": "", "min": "ABXXXX", "max": "ZLXX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Last Name"}, {"transformationsData": [{"feature_label": "Address", "transformation_label": "String Indexer"}], "feature": "Address", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3275", "mean": "", "stddev": "", "min": "10 MEADOW ROSE LN", "max": "9956 CLARK DR", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Address"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3275", "mean": "", "stddev": "", "min": "CO", "max": "CO", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Zip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3275", "mean": "80114.2", "stddev": "102.36", "min": "80002", "max": "80602", "missing": "0"}, "updatedLabel": "Zip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DriveTime", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3275", "mean": "13.31", "stddev": "7.38", "min": "0.65", "max": "30.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DriveTime"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Length of Residense", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3275", "mean": "10.94", "stddev": "10.76", "min": "0", "max": "55", "missing": "0"}, "updatedLabel": "Length of Residense"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MOR BANK: UPSCALE MERCHANDISE BUYER", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3275", "mean": "0.01", "stddev": "0.14", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "MOR BANK: UPSCALE MERCHAN..."}, {"transformationsData": [{"feature_label": "MOSAIC HOUSEHOLD", "transformation_label": "String Indexer"}], "feature": "MOSAIC HOUSEHOLD", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3275", "mean": "", "stddev": "", "min": "A01", "max": "U00", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MOSAIC HOUSEHOLD"}, {"transformationsData": [{"feature_label": "MOSAIC DESCRIPTION", "transformation_label": "String Indexer"}], "feature": "MOSAIC DESCRIPTION", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3275", "mean": "", "stddev": "", "min": "Aspirational Fusion", "max": "Young City Solos", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MOSAIC DESCRIPTION"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer_Lon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3275", "mean": "-104.92", "stddev": "0.12", "min": "-105.226518", "max": "-104.65312", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer_Lon"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer_Lat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3275", "mean": "39.69", "stddev": "0.09", "min": "39.433416", "max": "39.998058", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer_Lat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3275", "mean": "102.35", "stddev": "1.81", "min": "100", "max": "105", "missing": "0"}, "updatedLabel": "Store ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store_Lon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3275", "mean": "-104.92", "stddev": "0.12", "min": "-105.077754", "max": "-104.717928", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Store_Lon"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store_Lat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3275", "mean": "39.69", "stddev": "0.08", "min": "39.565799", "max": "39.856274", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Store_Lat"}, {"transformationsData": [{"feature_label": "Channel", "transformation_label": "String Indexer"}], "feature": "Channel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3275", "mean": "", "stddev": "", "min": "Catalog", "max": "eCommerce", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Channel"}, {"feature": "City_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3275", "mean": "1.57", "stddev": "2.28", "min": "0.0", "max": "17.0", "missing": "0"}, "updatedLabel": "City_stringindexer_transf..."}, {"feature": "Product_Category_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3275", "mean": "0.63", "stddev": "0.79", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Product_Category_stringin..."}, {"feature": "Product_Sub-Category_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3275", "mean": "5.1", "stddev": "4.26", "min": "0.0", "max": "16.0", "missing": "0"}, "updatedLabel": "Product_Sub-Category_stri..."}, {"feature": "Customer Segment_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3275", "mean": "1.23", "stddev": "1.11", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "Customer Segment_stringin..."}, {"feature": "First Name_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3275", "mean": "80.17", "stddev": "80.34", "min": "0.0", "max": "358.0", "missing": "0"}, "updatedLabel": "First Name_stringindexer_..."}, {"feature": "Last Name_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3275", "mean": "159.29", "stddev": "147.71", "min": "0.0", "max": "596.0", "missing": "0"}, "updatedLabel": "Last Name_stringindexer_t..."}, {"feature": "Address_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3275", "mean": "418.69", "stddev": "338.5", "min": "0.0", "max": "1279.0", "missing": "0"}, "updatedLabel": "Address_stringindexer_tra..."}, {"feature": "State_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3275", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "State_stringindexer_trans..."}, {"feature": "MOSAIC HOUSEHOLD_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3275", "mean": "15.4", "stddev": "13.54", "min": "0.0", "max": "60.0", "missing": "0"}, "updatedLabel": "MOSAIC HOUSEHOLD_stringin..."}, {"feature": "MOSAIC DESCRIPTION_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3275", "mean": "5.88", "stddev": "4.66", "min": "0.0", "max": "19.0", "missing": "0"}, "updatedLabel": "MOSAIC DESCRIPTION_string..."}, {"feature": "Channel_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3275", "mean": "0.49", "stddev": "0.68", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Channel_stringindexer_tra..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run customeracquisitionsHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=XG-Boost-classifier(autofe, ["City_stringindexer", "Product_Category_stringindexer", "Product_Sub-Category_stringindexer", "Count", "Customer ID", "Store Number", "Customer Segment_stringindexer", "First Name_stringindexer", "Last Name_stringindexer", "Address_stringindexer", "State_stringindexer", "Zip", "DriveTime", "Length of Residense", "MOR BANK: UPSCALE MERCHANDISE BUYER", "MOSAIC HOUSEHOLD_stringindexer", "MOSAIC DESCRIPTION_stringindexer", "Customer_Lon", "Customer_Lat", "Store ID", "Store_Lon", "Store_Lat"], "Channel_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
